# Recent Screenshots


In [9]:
import threading
import os
import requests
import time
import urllib.parse
import datetime
import ipywidgets as widgets
from IPython.core.display import display, HTML, clear_output

prefix = os.environ.get('API_PREFIX', 'http://192.168.45.91:90/intranet')

def update_thread(text, out):
    while True:
        try:
            r = requests.get(prefix + "/api/stats/crawler/recent-activity", timeout=5 )
        except:
            # Probably a time-out:
            pass
        if r.status_code == 200:
            html = '<p>Updated at: %s</p><div>' % datetime.datetime.now()
            #html += '<div style="position:absolute; background:#000; left:0; right:0; top:0; bottom:0; z-index:1000">'
            html += '<div>'
            for ss in r.json()['screenshots']:
                params = {'url': ss[0], 'target_date': ss[1]}
                url = prefix + "/api/screenshots/?" + urllib.parse.urlencode(params)
                html += '<a href="%s" style="float: left;"><img src="%s" style="max-height:120px;"/></a>' % (url,url)
            html += '</div>'
            out.append_display_data(HTML(html))
        else:
            out.append_display_data(HTML('ERROR at %s' % datetime.datetime.now()))

        out.clear_output(wait=True)
        time.sleep(10)
    
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(target=update_thread, args=("", out))
thread.start()
